# Load and Inspect Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_excel(r"C:\Users\dimma\OneDrive\Υπολογιστής\Ο.Π.Α\Supervised Learning\Εργασια Linear Regression\greece_housing_data.xlsx")
df.sample(10)

In [ ]:
df.info()                           #Checking the number of rows and columns, the data types and the non-null values

In [ ]:
column_list = list(df.columns)          #Checking and printing the Columns
print(column_list)

In [ ]:
#Dropping the columns we dont need
df = df.drop(columns=["Νομαρχία",'Δήμος Καλλικράτη','Δημοτικό ή Κοινοτικό Διαμέρισμα','Ένδειξη ΑΠΑΑ','Πλήθος Προσόψεων','Είδος Εμπράγματου δικαιώματος Κτίσματος','Ποσοστό Συνιδιοκτησίας Κτίσματος','Ειδικές Συνθήκες Ακινήτου','Επιφάνεια Οικοπέδου (σε τ.μ.)','Είδος Εμπράγματου δικαιώματος Οικοπέδου','Είδος Εμπράγματου δικαιώματος Οικοπέδου','Συνολική Επιφάνεια Κτισμάτων στο οικόπεδο','Ημερομηνία Συμβολαίου','Ποσοστό Συνιδιοκτησίας Οικοπέδου'])

In [ ]:
df.isnull().sum()        #Checking the null values. "Επιφάνεια Βοηθητικών Χώρων" has a very high number of null values.

In [ ]:
(df.select_dtypes(include='number')    #checking the mean vs median to find the proper value to impute in the null values. In most cases mean is much higher than the median an with large STD
   .agg(['mean', 'median', 'std'])     #means right skewed. So better impute with median except "Επιφάνεια Βοηθητικών Χώρων". We suspect outliers.
   .round(2)
   .T
   .assign(diff_mean_median = lambda x: x['mean'] - x['median'])
   .sort_values(by='diff_mean_median', key=abs, ascending=True))

In [ ]:
#Imputing the null numeric values with the median since it is the better choice
numeric_cols = ['Τίμημα Δικαιώματος', 'Tιμή Ζώνης', 'Eπιφάνεια Κύριων Χώρων (σε τ.μ.)', 'Έτος Κατασκευής']
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
#Imputing the 'Επιφάνεια Βοηθητικών Χώρων (σε τ.μ.)' with 0. We assume that this is not missing values but that many houses do not have such spaces. An experts opinion would be useful here.
df["Επιφάνεια Βοηθητικών Χώρων (σε τ.μ.)"] = df["Επιφάνεια Βοηθητικών Χώρων (σε τ.μ.)"].fillna(0)

In [ ]:
#Checking the feature Όροφος.. It has large null values and the "Y" should be replaced with -1.Also is dtype object.
df['Όροφος'].value_counts()


In [ ]:
df['Όροφος'] = df['Όροφος'].replace('Υ', -1)

In [ ]:
df['Όροφος'].isnull().sum()

In [ ]:
#We  deal wιτη the feature "Όροφος" which is identified as an  object convert it in to numeric
df['Όροφος'] = pd.to_numeric(df['Όροφος'], errors='coerce')
#We fill the NaN values wth the median
df['Όροφος'] = df['Όροφος'].fillna(df['Όροφος'].median())

In [ ]:
df.describe()   #Checking Some Basic Statistics

# Univariate Analysis

In [ ]:
df.columns

In [ ]:
#We create a function to use for each numeric feature

def histogram_boxplot(df, column, bins=8, color='green', figsize=(10, 7)):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize)

    # Histogram
    df[column].hist(bins=bins, ax=ax1, color=color, edgecolor='black')
    ax1.set_title(f'{column} - Histogram')

    # Boxplot
    df.boxplot(column=column, ax=ax2, vert=False)
    ax2.set_title(f'{column} - Boxplot')

    plt.tight_layout()
    plt.show()

In [ ]:
histogram_boxplot(df, 'Τίμημα Δικαιώματος', bins=200, color='green')

In [ ]:
histogram_boxplot(df, 'Έτος Κατασκευής', bins=200, color='blue')

In [ ]:
histogram_boxplot(df, 'Eπιφάνεια Κύριων Χώρων (σε τ.μ.)', bins=200, color='blue')

In [ ]:
histogram_boxplot(df, 'Tιμή Ζώνης', bins=200, color='blue')

# Handling the outliers in the features
As we saw in the boxplot anf histograms there some extreme  outliers almost in every numeric feature, and some of them make no sense. Perhaps better focus in the majority of the dataset which concerns housing. In these cases perhaps we should ask for the experts opinion to ask if these entries are mistakes.


In [ ]:
#We keep from 'Έτος Κατασκευής' the buildings after 1850.
df = df[(df['Έτος Κατασκευής'] >= 1850)]

In [ ]:
#We keep the buildings under 500 s.m.
df = df[(df['Eπιφάνεια Κύριων Χώρων (σε τ.μ.)'] <= 500)]

In [ ]:
df = df[(df['Tιμή Ζώνης'] <= 5000)]

# Handling the outliers in the target feature 
In the first histogram and boxplot for the target feature we detected extreme outliers. first we will detect the percentage of the outlier. Our new dataset from now on will be df_clean

In [ ]:
df_clean = df.copy()    #Our dataframe will be df_clean

In [ ]:
#We log the feature so we can get a better distribution
df_clean['Τίμημα_log'] = np.log1p(df_clean['Τίμημα Δικαιώματος'])

In [ ]:
Q1 = df_clean['Τίμημα_log'].quantile(0.25)
Q3 = df_clean['Τίμημα_log'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

#Detect the outliers
outliers_mask = (df_clean['Τίμημα_log'] < lower) | (df_clean['Τίμημα_log'] > upper)

# Count the % of the outliers
outliers_count = outliers_mask.sum()
total_count = len(df_clean)
outliers_percentage = (outliers_count / total_count) * 100

print(f"Number of outliers: {outliers_count}")
print(f"Percentage of outliers: {outliers_percentage:.2f}%")

# We keep the filtered dataframe
df_clean = df_clean[~outliers_mask]



In [ ]:
histogram_boxplot(df_clean, 'Τίμημα_log', bins=50, color='blue')

In [ ]:
#We clean the extreme outliers from the distribution
Q1 = df_clean['Τίμημα_log'].quantile(0.25)
Q3 = df_clean['Τίμημα_log'].quantile(0.75)
IQR = Q3 - Q1

# We set the boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# We keep the data inside the boundaries
df_clean = df_clean[(df_clean['Τίμημα_log'] >= lower_bound) & (df_clean['Τίμημα_log'] <= upper_bound)]

In [ ]:
histogram_boxplot(df_clean, 'Τίμημα_log', bins=50, color='blue')

# Converting the "Έτος Κατασκευής" Into " Ηλικία "

In [ ]:
#We create the feature "Ηλικία"
df_clean['Ηλικία'] = 2025 - df['Έτος Κατασκευής']

In [ ]:
#We drop column "Έτος Κατασκευής" and 'Τίμημα Δικαιώματος" to avoid Multicollinearity. For the Regression we have now "Τιμημα_log" as target column
df_clean = df_clean.drop(columns=['Έτος Κατασκευής'])

In [ ]:
df_clean = df_clean.drop(columns=['Τίμημα Δικαιώματος'])

# One Hot_Encoding in "Κατηγορία Ακινήτου"

In [ ]:
#Checking the values in the feature and creating a Barplot. We see that many Categories have very few entries. Perhaps bound them in one category
(df_clean['Κατηγορία Ακινήτου'].value_counts()
                                     .plot(kind='bar', figsize=(12,8), color=['red', 'blue'], edgecolor='black', width=0.8))
plt.title('Κατηγορία Ακινήτου', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Κατηγορία Ακινήτου', fontsize=6)
plt.ylabel('Αριθμός Ακινήτων', fontsize=12)
plt.xticks(rotation=90)
for i, val in enumerate(df_clean['Κατηγορία Ακινήτου'].value_counts()):
    plt.text(i, val , str(val), ha='center', va='bottom', fontsize=10, fontweight='bold')
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
#We set the boundary gor the categories that have less than 500 entries
counts = df_clean['Κατηγορία Ακινήτου'].value_counts()
threshold = 500
repl = counts[counts <= threshold].index
df_clean['Κατηγορία Ακινήτου'] = df_clean['Κατηγορία Ακινήτου'].replace(repl, 'Λοιπά Κτίρια')

In [ ]:
# We replace the categories now
df_clean['Κατηγορία Ακινήτου'] = df_clean['Κατηγορία Ακινήτου'].replace(repl, 'Λοιπά Κτίρια')

# Apply the One-Hot Encoding
df_clean = pd.get_dummies(df_clean, columns=['Κατηγορία Ακινήτου'], prefix='Cat', drop_first=True)

# Implementing The StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
# We separate the features from the target columns, since the target column is already logged
X = df_clean.drop(columns=['Τίμημα_log'])
y = df_clean['Τίμημα_log']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Implementing the Scaler in the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)

# Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

In [ ]:
y_train_pred_log = model.predict(X_train_scaled)
y_test_pred_log = model.predict(X_test_scaled)

In [ ]:
train_r2 = r2_score(y_train, y_train_pred_log)
r2_test = r2_score(y_test, y_test_pred_log)

In [ ]:
print(f"--- Linear Regression  ---")
print(f"R² in Train set: {train_r2:.6f}")
print(f"R² in Test set:  {r2_test:.6f}")

In [ ]:
y_test_pred_safe = np.clip(y_test_pred_log, y_train.min(), y_train.max())

In [ ]:
y_test_euro = np.expm1(y_test)
y_pred_euro = np.expm1(y_test_pred_safe)

In [ ]:
mae_euro = mean_absolute_error(y_test_euro, y_pred_euro)
rmse_euro = np.sqrt(mean_squared_error(y_test_euro, y_pred_euro))

print(f"\n--- Metrics in Prices  ---")
print(f"MAE in Prices:  {mae_euro:,.2f}")
print(f"RMSE in Prices: {rmse_euro:,.2f}")

In [ ]:
coefficients_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})

coefficients_df = coefficients_df.sort_values(by='Coefficient', ascending=False)

print("\n-- Linear Regression Coefficients (Scaled) --")
print(coefficients_df)
print(f"\nIntercept: {model.intercept_:.4f}")

In [ ]:
# 1. Παίρνουμε τις τυπικές αποκλίσεις από τον scaler
stds = scaler.scale_

# 2. Διαιρούμε τους scaled συντελεστές με τις τυπικές αποκλίσεις
unscaled_coefficients = model.coef_ / stds

# 3. Δημιουργία του DataFrame
unscaled_df = pd.DataFrame({
    'Feature': X.columns,
    'Unscaled Coefficient': unscaled_coefficients
})

# Ταξινόμηση
unscaled_df = unscaled_df.sort_values(by='Unscaled Coefficient', ascending=False)

print("-- Regression Coefficients (Original Scale) --")
print(unscaled_df)

# Apply Ridge Regression

In [ ]:
from sklearn.linear_model import RidgeCV

# We try different alpha values
ridge_cv = RidgeCV(alphas=[0.01, 0.1, 1.0, 10.0, 50.0, 100.0], scoring='neg_mean_absolute_error')
ridge_cv.fit(X_train_scaled, y_train)

In [ ]:
# The prediction
y_test_pred_log_ridge = ridge_cv.predict(X_test_scaled)
y_test_pred_safe_ridge = np.clip(y_test_pred_log_ridge, y_train.min(), y_train.max())

In [ ]:
# We transform into real money prices
y_pred_euro_ridge = np.expm1(y_test_pred_safe_ridge)
mae_euro_ridge = mean_absolute_error(y_test_euro, y_pred_euro_ridge)

In [ ]:
print(f"--- Ridge Cross Validation ---")
print(f"Be Alpha: {ridge_cv.alpha_}")
print(f"R² Test (Log):  {r2_score(y_test, y_test_pred_log_ridge):.4f}")
print(f"MAE in Price:     {mae_euro_ridge:,.2f} €")

# Visualizations

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test_euro, y_pred_euro, color='green', alpha=0.5)
max_val = max(y_test_euro.max(), y_pred_euro.max())
plt.plot([0, max_val], [0, max_val], color='red', linestyle='-')
plt.xlabel('Real Prices')
plt.ylabel('Models Predictions')
plt.title('Real Vs Models Predictions)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
#(Residuals = Actual - Predicted)
residuals = y_test - y_test_pred_log_ridge
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test_pred_log_ridge, y=residuals, alpha=0.5, color='green')
plt.axhline(y=0, color='red', linestyle='-')
plt.title('Residual Plot (Ridge Regression)')
plt.xlabel('Predicted Prices  (Log Scale)')
plt.ylabel('Residuals')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

# We put in the X axis the most powerfull Feature
sns.regplot(x=X_test['Eπιφάνεια Κύριων Χώρων (σε τ.μ.)'],
            y=y_test,
            scatter_kws={'alpha':0.3, 'color':'blue'},
            line_kws={'color':'red', 'label':'Linear Regression'})

plt.title('Surface VS Price (Log Scale)')
plt.xlabel('Surface in S.M')
plt.ylabel('Price (Log Scale)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Συμπεράσματα
Το μοντέλο παρουσιάζει μια μέτρια  ικανότητα πρόβλεψης με R^2=0,48  και το Μέσο Απόλυτο Σφάλμα Περίπου 39.500.
Οι μεταβλητές που φαίνεται να επηρεάζουν περισσότερο Θετικά στην Scaled Κλίμακα φαίνεται να η Επιφάνεια Κυρίων Χώρων με 0,427, η Τιμή Ζώνης με 0,416 , ενώ αρνητικά οι Αποθήκες, Θέσεις Στάθμευσης και η Ηλικία.

#### Η ερμηνεία των συντελεστών στην αρχική κλίμακα (Unscaled)

Επιφάνεια Κύριων Χώρων (0.0092): Κάθε επιπλέον τετραγωνικό μέτρο αυξάνει την αξία του ακινήτου κατά περίπου 0.92%.
Τιμή Ζώνης (0.00075): Για κάθε ευρώ αύξησης Τιμή Ζώνης, η τιμή του ακινήτου αυξάνεται κατά 0.075%.
Όροφος (0.096): Κάθε επιπλέον όροφος προσθέτει περίπου 9.6% στην αξία του ακινήτου.
Ηλικία (-0.009): Κάθε έτος παλαιότητας μειώνει την αξία του ακινήτου κατά περίπου 0.9%.
Αποθήκες / Πάρκινγκ: Οι πολύ χαμηλοί συντελεστές (περίπου -2.0) αντικατοπτρίζουν σημαίνουν ότι οι βοηθητικοί χώροι κοστίζουν σημαντικά λιγότερο ανά τ.μ. σε σύγκριση με τους χώρους κατοικίας.

### Εφαρμογή Ridge
Το χαμηλό alpha=0,01 μετά το Cross Validation δείχνει οτι οι μεταβλητές μας δεν εμφανίζουν multicollinearity δηλαδή δεν υπάρχει η ανάγκη έντονης ομαλοποίησης. Ακόμη το R^2 έμεινε σχετικά σταθερό ακόμα και πριν την εφαρμογή της Ridge άρα δεν υπήρχε overfitting. Παρόλα αυτά η εφαρμογή Ridge ήταν μια δικλείδα ασφαλείας.

*** Ο StandardScaler εφαρμόστηκε μετά το Train-Test split.




